# Make Slack Less Noisy 🔇

This notebook allows us to categorize and sum counts of automated workflow messages.  The goal is to identify and reduce frequent notifications leading to less noise and less notification blindness

In [1]:
import pandas as pd
import ast
import re
from datetime import datetime
from pathlib import Path
import json

In [7]:
START_DATE="2024-12-01"
NOW_DATE=datetime.now().strftime('%Y-%m-%d')

In [8]:
# Retrieved from slack admin console
# https://lightkeeperdev.slack.com/services/export

#directory = Path('./export/update_errors/')
directory = Path('C:/Users/SamMiller/Downloads/update_errors')
pattern = re.compile(r'\d{4}-\d{2}-\d{2}')
messages = []
files = [file for file in directory.glob('*.json') if pattern.search(file.stem)]
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        if isinstance(data, list):
            messages.extend(data)

len(messages)

2763

### Clean the data

In [23]:
# Fix the regex pattern in the clean_message function
def clean_message(message):
    clean = {}
    if not message.get('attachments'):
        return
    # Use raw string r'pattern' or double escape backslashes
    customer_search = re.search(r'http://(.*?)\.lightkeeperhq\.com\|.*', message['attachments'][0]['title'])
    if not customer_search:
        return
    customer = customer_search.group(1)
    head = message['text']
    body = message['attachments'][0]['text']
    dt = datetime.fromtimestamp(int(float(message['ts'])))
    isAtChannel = '<!channel>' in head
    
    return {
     'customer': customer,
     'head': head,
     'body': body,
     'datetime': dt,
     'isAtChannel': isAtChannel
    }

In [24]:
clean_messages = [clean_message(message) for message in messages]
clean_messages = [message for message in clean_messages if message]

In [25]:
clean_messages[1]

{'customer': 'lonepine',
 'head': 'T0 ArchiveTask has not completed for 2024-12-31',
 'body': '```An update will be sent once it has completed.```',
 'datetime': datetime.datetime(2025, 1, 1, 0, 0, 31),
 'isAtChannel': False}

In [26]:
len(clean_messages)

2756

In [27]:
df = pd.DataFrame(clean_messages)

In [28]:
df.dtypes

customer               object
head                   object
body                   object
datetime       datetime64[ns]
isAtChannel              bool
dtype: object

### Filter and enhance

In [29]:
# Filter by datetime
df = df.loc[(df['datetime'] > START_DATE) & (df['datetime'] < NOW_DATE)]

In [31]:
# Create a map of message types we can aggregate by
MESSAGE_TYPES = {
    'daily_update_sla_fail': '^<!channel>.*Daily.*has not completed.*',
    't0_update_sla_fail': '^<!channel>.*T0.*has not completed.*',
    'broker_update_sla_fail': '.*Broker UpdateTask has not completed.*',
    'daily_missing_files': '^Daily files missing for.*',
    'daily_received_files': '^All Daily required files have now been receive.*',
    't0_missing_files': '^T0 files missing for.*',
    't0_received_files': '^All T0 required files have now been received.*',
    'build_error': '.*/.build error.*',
    'daily_non_current': '.*Daily UpdateTask.build-confirm non-current.*',
    't0_non_current': '.*T0 UpdateTask.build-confirm non-current.*',
    'daily_update_sla_recovery': 'Daily.*has now completed.*',
    't0_update_sla_recovery': 'T0.*has now completed.*',
}

df['message_type'] = df['head'].apply(lambda x: ','.join([k for k, v in MESSAGE_TYPES.items() if re.match(v, x)]) or None)

In [32]:
df = df.dropna()

In [33]:
counts = df.groupby(['message_type','customer'])['head'].count()\
                    .reset_index().sort_values(['head'], ascending=False)
countsAtChannel = df.loc[df['isAtChannel']==True].groupby(['message_type','customer'])['head'].count()\
                    .reset_index().sort_values(['head'], ascending=False)

### @channel counts

In [34]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(countsAtChannel)

,message_type,customer,head
92,t0_update_sla_fail,tremblant,31
81,t0_update_sla_fail,hudsonbay,27
23,daily_update_sla_fail,cypressfunds,13
79,t0_update_sla_fail,contour,9
70,t0_update_sla_fail,altimeter,8
12,daily_update_sla_fail,altimeter,8
25,daily_update_sla_fail,fortress,7
11,daily_update_sla_fail,altapark,7
89,t0_update_sla_fail,slatepath,6
61,t0_non_current,maverickcap,5


### Non-alerting workflow counts

In [35]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(counts)

,message_type,customer,head
201,t0_missing_files,tremblant,71
244,t0_received_files,tremblant,69
241,t0_received_files,slatepath,68
198,t0_missing_files,slatepath,65
296,t0_update_sla_recovery,tremblant,53
288,t0_update_sla_recovery,hudsonbay,52
246,t0_received_files,vivaldi,48
231,t0_received_files,hudsonbay,45
236,t0_received_files,lonepine,35
228,t0_received_files,engle,33


### Latest Daily Recovery Times

In [36]:
dailyRecoveryMessages = df[df['message_type'] == 'daily_update_sla_recovery'] 
latestTimes = dailyRecoveryMessages.groupby(['customer', 'message_type'])['datetime'].max().reset_index()
display(latestTimes)

,customer,message_type,datetime
0,altimeter,daily_update_sla_recovery,2025-04-04 09:44:57
1,beta-1-aventail,daily_update_sla_recovery,2025-01-07 11:46:48
2,beta-tremblant,daily_update_sla_recovery,2025-01-02 09:10:34
3,corvex,daily_update_sla_recovery,2025-02-12 11:22:49
4,crestline,daily_update_sla_recovery,2025-04-09 12:47:56
5,cypressfunds,daily_update_sla_recovery,2025-03-03 09:52:56
6,eagle,daily_update_sla_recovery,2025-03-11 09:02:48
7,fortress,daily_update_sla_recovery,2025-03-06 13:11:49
8,goldentree,daily_update_sla_recovery,2025-02-03 08:49:30
9,gsam,daily_update_sla_recovery,2025-01-01 10:14:01


### Latest T0 Recovery Times

In [37]:
dailyRecoveryMessages = df[df['message_type'] == 't0_update_sla_recovery'] 
latestTimes = dailyRecoveryMessages.groupby(['customer', 'message_type'])['datetime'].max().reset_index()
display(latestTimes)

,customer,message_type,datetime
0,adage,t0_update_sla_recovery,2025-01-06 23:38:09
1,adelphi,t0_update_sla_recovery,2025-04-01 20:00:25
2,altimeter,t0_update_sla_recovery,2025-04-04 10:04:20
3,anomaly,t0_update_sla_recovery,2025-02-14 19:06:51
4,aregence,t0_update_sla_recovery,2025-01-27 21:04:29
5,avalonglobal,t0_update_sla_recovery,2025-03-24 00:00:33
6,beta-1-aventail,t0_update_sla_recovery,2025-01-02 20:25:31
7,braidwell,t0_update_sla_recovery,2025-04-07 18:32:41
8,contour,t0_update_sla_recovery,2025-03-14 21:04:07
9,conversant,t0_update_sla_recovery,2025-01-13 17:30:31


In [39]:
# Filter for missing files notifications
missing_files_df = df[df['message_type'].isin(['t0_missing_files', 'daily_missing_files'])].copy()

# Create notification type column from message_type using .loc
missing_files_df.loc[:, 'notification_type'] = missing_files_df['message_type'].apply(
    lambda x: 'T0' if 't0_' in x else 'Daily')

# Determine who gets notified (this is based on isAtChannel) using .loc
missing_files_df.loc[:, 'recipient'] = missing_files_df['isAtChannel'].apply(
    lambda x: 'All channel members' if x else 'Channel subscribers')

# Select and reorder columns as requested
result_df = missing_files_df[['customer', 'notification_type', 'recipient', 'datetime', 'head']]

# Sort by datetime for better organization
result_df = result_df.sort_values(['customer', 'notification_type', 'datetime'])

# Preview the result
print(f"Total missing files notifications: {len(result_df)}")
result_df.head(10)

Total missing files notifications: 583


,customer,notification_type,recipient,datetime,head
121,adage,T0,Channel subscribers,2025-01-02 22:15:15,T0 files missing for Adage
174,adage,T0,Channel subscribers,2025-01-03 22:15:15,T0 files missing for Adage
234,adage,T0,Channel subscribers,2025-01-06 22:15:15,T0 files missing for Adage
289,adage,T0,Channel subscribers,2025-01-07 22:15:15,T0 files missing for Adage
326,adage,T0,Channel subscribers,2025-01-08 22:15:15,T0 files missing for Adage
1779,adelphi,T0,Channel subscribers,2025-03-10 19:00:03,T0 files missing for adelphi
1810,adelphi,T0,Channel subscribers,2025-03-11 19:00:03,T0 files missing for adelphi
1850,adelphi,T0,Channel subscribers,2025-03-12 19:00:05,T0 files missing for adelphi
1882,adelphi,T0,Channel subscribers,2025-03-13 19:00:03,T0 files missing for adelphi
1918,adelphi,T0,Channel subscribers,2025-03-14 19:00:01,T0 files missing for adelphi


In [40]:
# Export to CSV
result_df.to_csv('missing_files_notifications.csv', index=False)
print("CSV file 'missing_files_notifications.csv' created successfully!")

CSV file 'missing_files_notifications.csv' created successfully!
